<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-1/blob/main/BusquedaPorCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Busqueda por CSV

In [1]:
!pip install seaborn==0.9.0

     |████████████████████████████████| 215kB 3.9MB/s 
  Found existing installation: seaborn 0.11.1
    Uninstalling seaborn-0.11.1:
      Successfully uninstalled seaborn-0.11.1


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!wget https://www.dropbox.com/s/chfwr4x475y3p1v/noticiasFotos.zip?dl=0 -O 2.zip
!unzip 2.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: 2/467124.jpg            
  inflating: 2/467130.jpg            
  inflating: 2/467131.jpg            
  inflating: 2/467132.jpg            
  inflating: 2/467135.jpg            
  inflating: 2/467136.jpg            
  inflating: 2/467138.jpg            
  inflating: 2/467139.jpg            
  inflating: 2/467140.jpg            
  inflating: 2/46715.jpg             
  inflating: 2/467156.jpg            
  inflating: 2/46716.jpg             
  inflating: 2/467161.jpg            
  inflating: 2/467163.jpg            
  inflating: 2/467164.jpg            
  inflating: 2/467169.jpg            
  inflating: 2/46717.jpg             
  inflating: 2/467170.jpg            
  inflating: 2/467173.jpg            
  inflating: 2/453565.jpg            
  inflating: 2/453569.jpg            
  inflating: 2/45357.jpg             
  inflating: 2/453572.jpg            
  inflating: 2/453576.jpg            
  inflating: 2/453577.jpg  

In [4]:
!wget https://www.dropbox.com/s/lw24ve7ndobd4y5/EmbeddingsFotos.csv?dl=0 -O EmbeddingsFotos.csv

--2021-05-28 14:17:01--  https://www.dropbox.com/s/lw24ve7ndobd4y5/EmbeddingsFotos.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lw24ve7ndobd4y5/EmbeddingsFotos.csv [following]
--2021-05-28 14:17:02--  https://www.dropbox.com/s/raw/lw24ve7ndobd4y5/EmbeddingsFotos.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca891595b3440d001889cd778b1.dl.dropboxusercontent.com/cd/0/inline/BPWNqOO_3K_EoeeJhlS24OkAc-u_xXx2B3EolAwx8gH8M3G-FUy03hMbUlchb8Iyx6fctCXctuoEzx49Eqi5XHgn7DGkSjDzjryIp194gVuNinIy1N3AFXz3oDWm2sjEx8ZlksJkOjevAOPvRNiZnjwp/file# [following]
--2021-05-28 14:17:02--  https://uca891595b3440d001889cd778b1.dl.dropboxusercontent.com/cd/0/inline/BPWNqOO_3K_EoeeJhlS24OkAc-u_xXx2B3EolAwx8gH8M3G-FUy03hMbUlchb8Iy

In [5]:
df = pd.read_csv('EmbeddingsFotos.csv')

In [6]:
df.head()

,NombreFoto,Embedding
0,462220.jpg 0,[[ 2.3611324 0.7700528 1.5315906 -1.38208...
1,462220.jpg 1,[[-0.18996963 0.47556552 0.3286136 1.53095...
2,44569.jpg 0,[[-0.67055297 -0.4772751 -0.05659456 0.97165...
3,44569.jpg 1,[[-8.7787104e-01 1.4074993e+00 -9.2881995e-01...
4,44569.jpg 2,[[ 0.55190855 0.47906682 -1.9257076 -0.58596...


Foto Entrada

In [7]:
!wget https://www.dropbox.com/s/2t7n2q8b8gidjsh/foto.zip?dl=0 -O foto.zip
!unzip foto.zip

--2021-05-28 14:17:03--  https://www.dropbox.com/s/2t7n2q8b8gidjsh/foto.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2t7n2q8b8gidjsh/foto.zip [following]
--2021-05-28 14:17:03--  https://www.dropbox.com/s/raw/2t7n2q8b8gidjsh/foto.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucabb0e746af276fbb3517412b96.dl.dropboxusercontent.com/cd/0/inline/BPUiL4DHFcHy-QZkqlPCZjHzjs4vLLu2sb3Xtuvu0EqXUztxsb5raO3ogPJuCfsDwexNMA6K3c8WWGI9MTPb2lUUKutxG_kxhUb3S4egW_-ootm9bmEbsvQCNrphVXlWHv0afkWrn6qQ8TfLa7p_M6nw/file# [following]
--2021-05-28 14:17:04--  https://ucabb0e746af276fbb3517412b96.dl.dropboxusercontent.com/cd/0/inline/BPUiL4DHFcHy-QZkqlPCZjHzjs4vLLu2sb3Xtuvu0EqXUztxsb5raO3ogPJuCfsDwexNMA6K3c8WWGI9MTPb2lUUKutxG_kxh

### Métodos necesarios

In [10]:
!wget https://www.dropbox.com/s/uji1ufb009ic3m0/mobilenet_graph.pb?dl=0 -O mobilenet_graph.pb
# Leer mobilenet_graph.pb
with tf.io.gfile.GFile('mobilenet_graph.pb','rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as mobilenet:
    tf.import_graph_def(graph_def,name='')

--2021-05-28 14:19:12--  https://www.dropbox.com/s/uji1ufb009ic3m0/mobilenet_graph.pb?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/uji1ufb009ic3m0/mobilenet_graph.pb [following]
--2021-05-28 14:19:13--  https://www.dropbox.com/s/raw/uji1ufb009ic3m0/mobilenet_graph.pb
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde1f77ffa2ea4b46927d2e5c48.dl.dropboxusercontent.com/cd/0/inline/BPVxJ4QYppqfFJFBiv4LQpBjhhoVHKPFaPhN_iD1gGvEPzOJSuQ1hsCbawONRz0e29ZXJWctNJtvAHgi7AHEfhL_r7rxhE5Mil76ox4t0ifJROXtcXZii2KAgNMj5QmSYc3HdVxTuvZXhj_8slJ8g601/file# [following]
--2021-05-28 14:19:13--  https://ucde1f77ffa2ea4b46927d2e5c48.dl.dropboxusercontent.com/cd/0/inline/BPVxJ4QYppqfFJFBiv4LQpBjhhoVHKPFaPhN_iD1gGvEPzOJSuQ1hsCbawONRz0e29Z

In [16]:
!wget https://www.dropbox.com/s/z0o1of3580smtwr/facenet_keras.h5?dl=0 -O facenet_keras.h5

facenet = load_model('facenet_keras.h5')

--2021-05-28 14:20:33--  https://www.dropbox.com/s/z0o1of3580smtwr/facenet_keras.h5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/z0o1of3580smtwr/facenet_keras.h5 [following]
--2021-05-28 14:20:34--  https://www.dropbox.com/s/raw/z0o1of3580smtwr/facenet_keras.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc575862097e67dd12afab0ba97a.dl.dropboxusercontent.com/cd/0/inline/BPVe8q7eGxXrX0cAZ-ey7u8Ge85WUATBEtkdJ-00h1f0mYrMNO20zEjL-aK7Ku79x-zyiAThXBVqM-VDFQuPflqgI56AZhTIPyxHFL9oaditoJptK5T5W065G_WNr-XZmCsD3U3MawCM1A_ufROv52L8/file# [following]
--2021-05-28 14:20:34--  https://uc575862097e67dd12afab0ba97a.dl.dropboxusercontent.com/cd/0/inline/BPVe8q7eGxXrX0cAZ-ey7u8Ge85WUATBEtkdJ-00h1f0mYrMNO20zEjL-aK7Ku79x-zyiAThX

In [103]:
def load_image(DIR, NAME):
    return cv2.cvtColor(cv2.imread(f'{DIR}/{name}'), cv2.COLOR_BGR2RGB)

def detect_faces(image, score_threshold=0.7):
    global boxes, scores
    (imh, imw) = image.shape[:-1]
    img = np.expand_dims(image,axis=0)
    
    # Inicializar mobilenet
    sess = tf.compat.v1.Session(graph=mobilenet)
    image_tensor = mobilenet.get_tensor_by_name('image_tensor:0')
    boxes = mobilenet.get_tensor_by_name('detection_boxes:0')
    scores = mobilenet.get_tensor_by_name('detection_scores:0')
    
    # Predicción (detección)
    (boxes, scores) = sess.run([boxes, scores], feed_dict={image_tensor:img})
    
    # Reajustar tamaños boxes, scores
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)
    
    # Depurar bounding boxes
    idx = np.where(scores>=score_threshold)[0]
    
    # Crear bounding boxes
    bboxes = []
    for index in idx:
        ymin, xmin, ymax, xmax = boxes[index,:]
        (left, right, top, bottom) = (xmin*imw, xmax*imw, ymin*imh, ymax*imh)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        bboxes.append([left,right,top,bottom])
        
    return bboxes


def extract_faces(image,bboxes,new_size=(160,160)):
    cropped_faces = []
    for box in bboxes:
        left, right, top, bottom = box
        face = image[top:bottom,left:right]
        cropped_faces.append(cv2.resize(face,dsize=new_size))
    return cropped_faces

def compute_embedding(model,face):
    face = face.astype('float32')
    
    mean, std = face.mean(), face.std()
    face = (face-mean) / std
    
    face = np.expand_dims(face,axis=0)
    
    embedding = model.predict(face)
    return embedding

def compare_faces(embs_ref, emb_desc, umbral=14.9):
    distancias = []
    for emb_ref in embs_ref:
        distancias.append(np.linalg.norm(emb_ref-emb_desc))
    distancias = np.array(distancias)
    return distancias, list(distancias<=umbral)

In [36]:
known_embeddings = []

print('Procesando rostros conocidos...')
for name in os.listdir('foto'):
    if name.endswith('.jpg'):
        print(f'   {name}')
        image = load_image('foto',name)
        bboxes = detect_faces(image)
        face = extract_faces(image,bboxes)
        known_embeddings.append(compute_embedding(facenet,face[0]))

Procesando rostros conocidos...
   conchaAndreu.jpg


Comparación de embeddings

In [18]:
embeddingsModelo = df['Embedding']

In [51]:
embeddingsModelo[1]

'[[-0.18996963  0.47556552  0.3286136   1.5309514   0.62813014 -0.13005841\n   0.10592152 -0.66462034 -0.6861267  -0.6047762   0.6668252  -0.5080282\n   1.4240074   0.6851475   1.7242337   0.5311054  -0.5076395  -1.4917814\n  -1.4583507   0.88369215 -0.5048027  -0.03305591 -0.10728723  0.3505311\n   0.18363258 -0.60061824  0.13972794  0.84549934 -2.3456595   0.5540586\n  -0.17170511 -0.88935643 -0.74024487 -0.27362347 -0.26249826  1.2041694\n  -1.7959503  -0.39499325 -0.8599583   1.4153278   1.244695   -0.57961404\n   0.7124711  -0.47363117  0.913836   -1.6095527   2.4872813   0.7789227\n   1.4479051   0.1625808   0.5713562  -0.21613742  0.94967526 -2.4467711\n   0.1285314  -1.7116238   1.0727617   0.9707922   0.76895183  1.925824\n   0.5498862  -0.92531645  0.11586294 -1.1201551   0.40122917 -3.191715\n   0.09011545 -0.9434534  -1.0525503   1.3918947   0.8021957  -0.9103023\n   0.11099075  0.8200471  -0.74979705 -0.44416964  1.6260998   1.9299772\n  -0.10566363  0.41638696  0.01357811

In [ ]:
lista=[]
lista=embeddingsModelo[1].replace('[','').replace('\n','').replace(']','').split(' ')

In [ ]:
result = []
for item in lista:
    if item not in result:
        result.append(item)

In [ ]:
result

In [135]:
known_embeddings

[array([[ 0.16271353,  0.4862124 ,  1.8757727 , -0.8237939 ,  0.6594086 ,
         -0.36294362,  0.3353369 ,  0.77774036, -0.45129308, -0.54896337,
         -0.06453086,  0.08706567,  1.703111  ,  0.902349  ,  0.16001534,
         -1.4287717 , -0.162166  , -0.4813186 ,  0.78640175, -0.20852251,
         -0.9052143 , -0.08942164, -0.47549364,  0.58584034, -0.59641886,
          0.20514849, -0.85136545,  1.3090636 ,  0.48061806, -0.04805928,
         -0.77661455, -0.58808756,  0.75972855, -0.74531657,  0.01866588,
          0.5536681 , -0.24610786,  0.09339406, -0.96917206,  1.5265123 ,
          1.5485711 ,  1.0119599 ,  0.30268377,  0.30181766, -2.2646077 ,
          0.1828328 , -0.99502903, -0.07626627,  0.24588498, -0.5042027 ,
          1.910269  ,  2.288811  ,  0.3953287 , -2.9916039 ,  0.11267671,
          1.062661  ,  0.3110896 ,  0.3967503 ,  0.11094791, -0.6404621 ,
          0.96214145, -1.7445433 ,  0.75485   ,  1.3502483 , -1.2542285 ,
         -0.41722918, -0.09038687, -0.

In [134]:
lista=embeddingsModelo[1].replace('[','').replace('\n','').replace(']','').split(' ')
lista
aux=[]
for elem in lista:
  if elem != '':
    aux.append(float(elem))


li=[]
li.append(' '.join(aux))

li


TypeError: ignored

In [123]:
 print(known_embeddings[0])

[[ 0.16271353  0.4862124   1.8757727  -0.8237939   0.6594086  -0.36294362
   0.3353369   0.77774036 -0.45129308 -0.54896337 -0.06453086  0.08706567
   1.703111    0.902349    0.16001534 -1.4287717  -0.162166   -0.4813186
   0.78640175 -0.20852251 -0.9052143  -0.08942164 -0.47549364  0.58584034
  -0.59641886  0.20514849 -0.85136545  1.3090636   0.48061806 -0.04805928
  -0.77661455 -0.58808756  0.75972855 -0.74531657  0.01866588  0.5536681
  -0.24610786  0.09339406 -0.96917206  1.5265123   1.5485711   1.0119599
   0.30268377  0.30181766 -2.2646077   0.1828328  -0.99502903 -0.07626627
   0.24588498 -0.5042027   1.910269    2.288811    0.3953287  -2.9916039
   0.11267671  1.062661    0.3110896   0.3967503   0.11094791 -0.6404621
   0.96214145 -1.7445433   0.75485     1.3502483  -1.2542285  -0.41722918
  -0.09038687 -0.19936253  2.135515   -0.22333734  0.80784816 -0.9867066
   0.317428   -0.72932136  1.1823704   0.6617123   0.5134462   1.0692954
  -1.9696817   0.19821273  0.06431494  1.5651

In [116]:
def compare_faces(embs_ref, emb_desc, umbral=15.56):
    distancias = []
    for emb_ref in embs_ref:
        distancias.append(np.linalg.norm(emb_ref-emb_desc))
    distancias = np.array(distancias)
    return distancias, list(distancias<=umbral)

In [138]:
min=14
max=0
win=''
while min<17:
  cont=0
  for caraEmb in embeddingsModelo:
    lista=embeddingsModelo[1].replace('[','').replace('\n','').replace(']','').split(' ')
    result = []
    for item in lista:
      if item not in result:
        result.append(item)
    result.remove('')
    aux=[]
    for elem in result:
      aux.append(float(elem))
    li=[]
    li.append(aux)

    _, reconocimiento = compare_faces(known_embeddings,li)
    if any(reconocimiento):
      #print('     match!')
      cont=cont+1
  if cont<max:
    max=cont
    win=str(cont)+str(min)
  min=min+0.00
  



In [136]:
cont=0
for caraEmb in embeddingsModelo:
  lista=embeddingsModelo[1].replace('[','').replace('\n','').replace(']','').split(' ')
  result = []
  for item in lista:
    if item not in result:
      result.append(item)
  result.remove('')
  aux=[]
  for elem in result:
    aux.append(float(elem))
  li=[]
  li.append(aux)
 
  _, reconocimiento = compare_faces(known_embeddings,li)
  if any(reconocimiento):
    #print('     match!')
    cont=cont+1

cont

10290